In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import analysis

## Run code on qlua output

In [ ]:
cfgbase = 'cl3_16_48_b6p1_m0p2450'

# some nice looking momentum inversion data.
# job_num = 16142
# mom_list = []
# for i in range(1, 8 + 1):
#     for j in range(2, 10 + 1, 2):
#         if j + 1 >= i and not (i == 8 and j == 10):
#             mom_list.append([i, i, i, j])

# job_num = 16165
# mom_list = []
# for i in range(1, 16 + 1):
#     for j in range(i, 16 + 1):
#         if j % 2 == 0:
#             mom_list.append([i, i, i, j])

job_num = 17680
# job_num = 17809
# When we do the momentum inversion, Z(p) is purely real (although large). We also 
mom_list = [[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4], [5, 5, 5, 5]]

analysis.mom_list = mom_list
analysis.mom_str_list = [analysis.plist_to_string(p) for p in mom_list]

base = './output/' + cfgbase + '_' + str(job_num)
# props, threepts = analysis.readfile(base)
props, threepts = analysis.readfile(base, dpath = 'prop1/')

V = analysis.get_hypervolume()

In [ ]:
mom_list = analysis.get_mom_list()
s = 0    # TODO seed this with something random
props_boot = analysis.bootstrap(props, seed = s)
threept_boot = analysis.bootstrap(threepts, seed = s)

In [ ]:
props_inv = analysis.invert_prop(props_boot)
Γ = analysis.amputate(props_inv, threept_boot)
Zq = analysis.quark_renorm(props_inv)
Γ_B, Γ_B_inv = analysis.born_term()

In [ ]:
Z = analysis.get_Z(Zq, Γ, Γ_B_inv)
# Z = analysis.get_Z(Zq, Γ, Γ_B)

In [ ]:
#compare with analysis_EXAMPLE.py for p2222 and cfg 200
print('Two point')
print(props_boot['p2222'][0, 0])
print('Two point inverse')
print(props_inv['p2222'][0, 0])
print('Quark renormalization')
print(Zq['p2222'])

In [ ]:
μ, σ = analysis.get_statistics_Z(Z)
print(μ)
print(σ)

In [ ]:
p_squared = np.array([np.abs(analysis.square(p)) for p in mom_list])
μ_list = np.array([μ[analysis.plist_to_string(p)] for p in mom_list])
σ_list = np.array([σ[analysis.plist_to_string(p)] for p in mom_list])
plt.figure()
plt.errorbar(p_squared, np.real(μ_list), yerr = np.abs(σ_list), fmt = 'o')
# for i in range(len(p_squared)):
#     plt.text(p_squared[i], np.abs(μ_list[i]), analysis.mom_list[i])
plt.ylabel('$\mathcal{Z}(p^2)$')
plt.xlabel('$p^2$')
plt.title('Renormalization of quark operator')
plt.show()

In [ ]:
for p, psquared in zip(mom_list, p_squared):
    print(p, psquared)

In [ ]:
# plt.savefig('/Users/theoares/lqcd/npr_momfrac/output/cl3_16_48_b6p1_m0p2450_16142/renorm_constant_zoomed_error.pdf')

In [ ]:
# cfg_list, err, means = analysis.error_analysis(Z, 4, 4)

In [ ]:
# plt.figure()
# plt.scatter(cfg_list, err)
# plt.title('Error scaling at p = [2, 2, 2, 2]')
# plt.xlabel('Number configurations')
# plt.ylabel('Error')
# plt.ylim((.0004, .002))
# plt.show()